In [10]:
import sys
sys.path.append('../')
import utility as util

config = util.load_specific_yaml('instance_F_1.yaml')
# print(config)
print(config['V'])
print(config['j_amount'])
print(config['k_amount'])

[[118429, 206843, 228891], [304506, 272310, 271326], [150915, 56261, 206380], [273035, 25288, 131093], [269547, 202219, 316429], [293500, 72203, 149248], [385467, 469837, 106747], [79615, 173705, 105400], [489018, 471679, 247909], [453965, 35130, 143000], [161513, 481068, 18194], [264545, 287004, 344100], [37815, 234839, 365419], [56699, 116409, 323932], [252101, 305304, 269387], [193266, 158863, 342020], [53664, 331874, 307457], [487637, 324810, 183427], [299385, 233792, 287873], [161098, 312557, 283906]]
20
3


In [16]:
def E_function():
	pass
def G_function():
	pass
def greedy_best_location(config: dict, candidates: list):
	best_util = 0
	# Find the best location with most utility.
	for ind, facility in enumerate(candidates):
		if facility != 0:
			# start filling with largest V
			car_list = [(value, index) for index, value in enumerate(config['V'][ind])]
			sorted_car_list = sorted(car_list, reverse=True)
			
			
			

In [5]:
# heuristic_greedy
"""
進步中 = true
最佳W = 0
已分配設施 = {...都是0}
while 此輪還在進步 and 還有工廠能蓋
    進步中 = false
    此輪最佳W 暫時= 最佳W（前一輪的）
    對每個可能的建立工廠點 j:
        看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，
        剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿E
        計算總W (包含先前iter已蓋好之工廠點)
        如果大於此輪最佳，記錄為此輪最佳，更新最佳蓋哪個點 j
    如果此輪最佳obj > 最佳obj:
        更新已分配設施用這個最佳設廠點j跟已分配設施
        更新最佳obj
"""
improve = True
best_obj = 0
candidates = [0 for i in range(config['j_amount'])]
while improve:
	loc_to_build = greedy_best_location(config, candidates)
	cur_obj = 0
			
	if cur_obj > best_obj:
        best_obj = cur_obj
        best_facility = ind

TabError: inconsistent use of tabs and spaces in indentation (2648499688.py, line 27)

In [18]:
def sort_with_index(lst):
    indexed_list = [(value, index) for index, value in enumerate(lst)]
    sorted_list = sorted(indexed_list, reverse=True)
    sorted_indices = [index for value, index in sorted_list]
    sorted_values = [value for value, index in sorted_list]
    return sorted_values, sorted_indices

# Example usage:
my_list = [3, 6, 1, 9, 2]
sorted_values, sorted_indices = sort_with_index(my_list)
print("Sorted values:", sorted_values)
print("Original indices of sorted values:", sorted_indices)


Sorted values: [9, 6, 3, 2, 1]
Original indices of sorted values: [3, 1, 0, 4, 2]
